#### OpenaSparkNotebookandcreateasparkbasedataframebyreadingallthefilesunder/public/sms/usersfolder.Docheckhowmanypartitionsarecreatedinyourdataframe.

In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()

spark = SparkSession \
.builder \
.config('spark.ui.port', '0') \
.config('spark.shuffle.useOldFetchProtocol', 'true') \
.enableHiveSupport() \
.master('yarn') \
.getOrCreate()

In [2]:
spark

In [3]:
from pyspark.sql.types import *
users_schema = StructType([
    StructField("user_id", IntegerType(), nullable=False)
    , StructField("user_first_name", StringType(), nullable = False)
    , StructField("user_last_name", StringType(), nullable = False)
    , StructField("user_email", StringType(), nullable = False)
    , StructField("user_gender", StringType(), nullable = False)
    , StructField("user_phone_numbers", ArrayType(StringType()), nullable = True)
    , StructField("user_address", StructType([
        StructField("street", StringType(), nullable = False)
        , StructField("city", StringType(), nullable = False)
        , StructField("state", StringType(), nullable = False)
        , StructField("postal_code", StringType(), nullable = False)
    ]), nullable = False)
])

In [4]:
users_df = spark.read \
.format("json") \
.schema(users_schema) \
.load("/public/sms/users")

In [5]:
users_df.show()

+-------+---------------+--------------+--------------------+-----------+--------------------+--------------------+
|user_id|user_first_name|user_last_name|          user_email|user_gender|  user_phone_numbers|        user_address|
+-------+---------------+--------------+--------------------+-----------+--------------------+--------------------+
| 200001|         Eirena|     Cutsforth|ecutsforth0@wisc.edu|     Female|[4197404036, 9173...|{8 Warrior Drive,...|
| 200002|          Marja|      Shopcott|mshopcott1@hexun.com|     Female|[9542037028, 2128...|{66 Prairieview T...|
| 200003|           Dawn|       Tointon|  dtointon2@ucsd.edu|     Female|[9523035647, 2134...|{18 Ronald Regan ...|
| 200004|          Goldi|        Leaman|     gleaman3@360.cn|     Female|[2027069459, 7042...|{7696 Calypso Jun...|
| 200005|       Brewster|      Hallagan|bhallagan4@livejo...|       Male|[8134746319, 2152...|{942 Emmet Park, ...|
| 200006|       Florence|       Glashby|fglashby5@deviant...|     Female

In [6]:
users_df.rdd.getNumPartitions()

3

In [7]:
users_df.count()

1000000

In [8]:
users_df.filter("user_address.state = 'New York'").count()

49576

In [9]:
from pyspark.sql.functions import *
num_of_postal_codes_df = users_df.groupBy("user_address.state").agg(countDistinct("user_address.postal_code").alias('num_of_postal_codes'))

In [10]:
num_of_postal_codes_df.orderBy(desc("num_of_postal_codes")).show(1)

+----------+-------------------+
|     state|num_of_postal_codes|
+----------+-------------------+
|California|                206|
+----------+-------------------+
only showing top 1 row



In [11]:
num_of_max_users_df = users_df.filter("user_address.city is not null").groupBy("user_address.city").agg(countDistinct("user_id").alias("num_of_users"))

In [12]:
num_of_max_users_df.orderBy(desc("num_of_users")).show(1)

+----------+------------+
|      city|num_of_users|
+----------+------------+
|Washington|       28504|
+----------+------------+
only showing top 1 row



In [13]:
bizjournal_users_df = users_df.select('user_id').where("user_email like '%bizjournals.com'")

In [14]:
bizjournal_users_df.distinct().count()

2015

In [16]:
users_df.select("user_id").where("size(user_phone_numbers) = 4").distinct().count()

179041

In [17]:
users_df.select("user_id").where("size(user_phone_numbers) = -1").distinct().count()

108981

In [19]:
users_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv009538/week9/project") \
.save()